In [23]:
import pandas as pd
from itertools import combinations
import networkx as nx
from scipy.stats import spearmanr, kendalltau
import numpy as np
import re
import copy

# Women

## Gossip Girl

In [24]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_Gossip_Girl.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/gossip_girl_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_00", "Eric")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_01", "Blair")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_02", "Dan")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_03", "Serena")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_04", "Blair")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_05", "Jenny")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_06", "Serena")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_07", "Rufus")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_08", "Lily")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_09", "Various/Mistakes")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_12", "Nate")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_13", "Serena")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_14", "Blair")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_16", "Chuck")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_18", "Chuck")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_19", "Eleanor")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_20", "Gossip_Girl")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_21", "Mr_Van_der_Bilt")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_22", "Dan")
  return df

df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

In [26]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

27

In [9]:
def get_correlations(df, ratings):
    groups = df.groupby("Episode_order")

    occurrences = []

    for epi in groups.groups:
        ep = groups.get_group(epi)
        occ = ep.Speaker.values
        occurrences.append(occ)


    lst = []

    for occ in occurrences:
        dct = {}
        G = nx.Graph()
        si, ei = 0, 2
        while ei < len(occ):
            interaction = []
            for character in set(occ[si : ei]):
                interaction.append(character)
            interaction = set(interaction)
            if len(interaction) > 1:
                for sp1, sp2 in combinations(interaction, 2):
                    if G.has_edge(sp1, sp2):
                        G[sp1][sp2]['weight'] += 1
                    else:
                        G.add_edge(sp1, sp2, weight=1)
            si += 1
            ei += 1

        
        weighted = nx.pagerank(G) 
        char = {c: weighted[c] for c in G.nodes}
        lst.append(char)

    chardf = pd.DataFrame(lst)

    chars = [col for col in chardf.columns if len(chardf[col].dropna()) >= 1]

    main = chardf[chars]
    main = main.fillna(0)
    ratings = ratings.join(main)

    char_plus = chars + ["rating", "total_votes"]
    chardf = ratings[char_plus]
    rho, pval = spearmanr(chardf)

    corr = pd.DataFrame(rho, columns=chardf.columns, index=chardf.columns)
    pvaldf = pd.DataFrame(pval, columns=chardf.columns, index=chardf.columns)


    # RATINGS
    pvald_stars = dict(zip(chardf.columns, pvaldf['rating']))

    app_stars = corr[['rating', 'total_votes']]
    app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
    app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


    rho_stars = app_stars.drop(columns=['total_votes']).fillna(0).sort_values(by='rating', ascending=False, key=np.abs)


    # TOTAL VOTES
    pvald_votes = dict(zip(chardf.columns, pvaldf['total_votes']))

    app_votes = corr[['rating', 'total_votes']]

    rho_votes = app_votes.drop(columns=['rating']).fillna(0).sort_values(by='total_votes', ascending=False, key=np.abs)
    rho_votes['pval'] = [pvald_votes[t] for t in rho_votes.index]
    rho_votes['significant?'] = [True if value < 0.05 else False for value in rho_votes.pval.values]


    return rho_stars, rho_votes


In [10]:
rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [11]:
rho_stars

,rating,pval,significant?
rating,1.000000,0.000000,True
Various/Mistakes,-0.307639,0.044767,True
SPEAKER_26,0.297109,0.053014,False
Blair,0.296041,0.053915,False
SPEAKER_24,0.290767,0.058544,False
Dan,-0.258807,0.093779,False
Mr_Van_der_Bilt,-0.229696,0.138414,False
Serena,-0.219437,0.157409,False
Nate,-0.206390,0.184226,False
SPEAKER_10,-0.191688,0.218173,False


In [12]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,0.000000,True
SPEAKER_11,-0.439977,0.003154,True
Eric,0.341778,0.024885,True
Dan,0.339060,0.026136,True
SPEAKER_25,0.331445,0.029918,True
SPEAKER_10,-0.325931,0.032932,True
SPEAKER_15,0.239305,0.122215,False
Serena,0.229111,0.139449,False
Mr_Van_der_Bilt,-0.222046,0.152406,False
Jenny,0.220276,0.155788,False


## Bridgerton

In [27]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_Bridgerton.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/bridgerton_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_00", "Colin/Music")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_01", "Lady_Danbury")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_02", "Genevieve")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_03", "Anthony")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_04", "Music")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_05", "Daphne")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_07", "Portia")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_08", "Kate")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_10", "Violet_Bridgerton")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_11", "Lady_Cowper/Cressida/Mary_Sharma")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_12", "Edwina")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_13", "Narrator")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_16", "Daphne")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_17", "Simon")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_18", "Benedict")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_19", "Penelope")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_21", "Marina")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_22", "Anthony")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_23", "emotional_utterances")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_24", "Eloise")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_25", "Jack")
  return df

df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [28]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

26

In [14]:
rho_stars

,rating,pval,significant?
rating,1.000000,0.000000,True
Benedict,0.689107,0.003150,True
Narrator,-0.672700,0.004298,True
Simon,-0.580222,0.018459,True
Jack,0.546218,0.028594,True
Kate,0.520601,0.038685,True
SPEAKER_06,-0.516085,0.040712,True
Daphne,-0.504152,0.046449,True
total_votes,-0.498186,0.049534,True
Edwina,0.411674,0.113121,False


In [15]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,0.000000,True
Edwina,-0.894118,0.000003,True
Kate,-0.796464,0.000222,True
Simon,0.794118,0.000239,True
Jack,-0.767428,0.000520,True
Daphne,0.761765,0.000605,True
Benedict,-0.732353,0.001255,True
SPEAKER_14,0.688742,0.003172,True
Music,0.558824,0.024437,True
SPEAKER_20,0.555882,0.025362,True


## Pretty Little Liars

In [29]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_Pretty_Little_Liars.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/pretty_little_liars_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_02", "Spencer")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_04", "Paige")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_06", "music")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_11", "Caleb")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_17", "Ezra")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_19", "Mona")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_20", "Aria")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_23", "Emily")
  return df

df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [30]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

27

In [31]:
rho_stars

,rating,pval,significant?
rating,1.000000,0.000000,True
SPEAKER_16,-0.546756,0.000070,True
SPEAKER_21,0.344064,0.017881,True
Paige,0.307179,0.035704,True
SPEAKER_00,0.290424,0.047665,True
SPEAKER_03,-0.265729,0.071027,False
SPEAKER_01,-0.253597,0.085432,False
Mona,0.242311,0.100792,False
SPEAKER_09,0.226719,0.125402,False
SPEAKER_05,0.214793,0.147093,False


In [35]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,0.000000e+00,True
SPEAKER_21,-0.706852,2.809570e-08,True
SPEAKER_16,0.542180,8.269480e-05,True
music,0.403180,4.954281e-03,True
Spencer,-0.373345,9.743281e-03,True
SPEAKER_18,-0.323388,2.660300e-02,True
SPEAKER_05,0.251055,8.872105e-02,False
Paige,-0.249668,9.055762e-02,False
SPEAKER_07,0.232611,1.156204e-01,False
SPEAKER_10,-0.216016,1.447481e-01,False


## Jane the Virgin

In [32]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_Jane_the_Virgin.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/jane_the_virgin_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_01", "Rogelio")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_04", "Rogelio")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_08", "Dina/Xiomara")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_10", "Rafael")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_15", "Jane")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_18", "Michael")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_19", "Narrator")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_24", "Petra")
  return df

df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/home/ada/anaconda3/envs/Python/lib/python3.8/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "
/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] =

In [33]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

26

In [38]:
rho_stars

,rating,pval,significant?
rating,1.000000,0.000000,True
total_votes,0.555037,0.000092,True
SPEAKER_13,-0.529018,0.000223,True
SPEAKER_11,0.451331,0.002105,True
SPEAKER_17,-0.335145,0.026158,True
SPEAKER_05,-0.334067,0.026674,True
Michael,0.331049,0.028162,True
SPEAKER_14,0.293899,0.052821,False
SPEAKER_07,-0.270545,0.075697,False
SPEAKER_09,0.258760,0.089880,False


In [39]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,0.000000e+00,True
SPEAKER_06,-0.722864,3.005201e-08,True
SPEAKER_13,-0.603961,1.420623e-05,True
SPEAKER_07,-0.599662,1.695301e-05,True
Rafael,0.557161,8.553767e-05,True
rating,0.555037,9.221770e-05,True
SPEAKER_11,0.491893,6.958594e-04,True
SPEAKER_12,0.449182,2.223899e-03,True
Jane,0.352974,1.876492e-02,True
Michael,0.343953,2.224919e-02,True


In [ ]:
# rho stars average significant characters = 4.25 (1+7+4+5 /4)
# rho votes average significant characters = 8.5 (5+12+5+12 /4)

# Men

## Spartacus

In [34]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_Spartacus.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/spartacus_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_02", "Glaber")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_03", "Lucretia")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_11", "Oenomaus")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_14", "Asher")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_15", "Ilithyia")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_17", "Gannicus")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_19", "Spartacus")
  return df

df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [35]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

25

In [42]:
rho_stars 

,rating,pval,significant?
rating,1.000000,0.000000,True
SPEAKER_20,-0.399539,0.021248,True
total_votes,0.389142,0.025201,True
Glaber,-0.347003,0.047875,True
SPEAKER_24,0.307835,0.081369,False
SPEAKER_06,-0.250357,0.159952,False
SPEAKER_04,0.198576,0.267936,False
SPEAKER_13,-0.188019,0.294720,False
SPEAKER_09,-0.163187,0.364198,False
SPEAKER_22,-0.155434,0.387730,False


In [43]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,0.000000,True
SPEAKER_12,-0.567059,0.000580,True
SPEAKER_23,-0.548023,0.000962,True
SPEAKER_09,0.509859,0.002438,True
SPEAKER_05,-0.495678,0.003354,True
SPEAKER_08,0.439241,0.010545,True
Spartacus,-0.420719,0.014768,True
Gannicus,-0.420323,0.014872,True
SPEAKER_01,-0.413972,0.016623,True
Lucretia,0.410295,0.017713,True


## Narcos

In [36]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_Narcos.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/narcos_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_00", "Gustavo/Jorge")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_04", "Amado")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_05", "Javier")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_06", "Jorge")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_07", "Maria/Paola")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_08", "Miguel_Rodriguez")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_11", "Pacho")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_14", "Carlo/David")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_15", "Feistl/Gustavo")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_16", "Van_Ness")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_17", "Narrator")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_18", "Gilberto_Rodriguez")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_23", "Bill_Stetchner")
  return df


df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [37]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

26

In [50]:
rho_stars

,rating,pval,significant?
rating,1.000000,0.000000,True
Gustavo/Jorge,0.507658,0.004187,True
Feistl/Gustavo,-0.378956,0.038907,True
SPEAKER_10,-0.329053,0.075809,False
Carlo/David,0.280865,0.132724,False
SPEAKER_02,-0.266118,0.155191,False
total_votes,0.214056,0.256022,False
SPEAKER_25,0.211152,0.262684,False
Maria/Paola,-0.207353,0.271566,False
Narrator,-0.170038,0.369009,False


In [51]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,0.000000e+00,True
SPEAKER_09,0.772577,5.667342e-07,True
Miguel_Rodriguez,-0.728731,4.962451e-06,True
SPEAKER_22,0.721994,6.678845e-06,True
Jorge,-0.716654,8.401447e-06,True
Van_Ness,0.678611,3.757999e-05,True
SPEAKER_24,0.579978,7.814169e-04,True
Javier,-0.571524,9.700662e-04,True
Gilberto_Rodriguez,-0.530649,2.555218e-03,True
SPEAKER_12,0.529700,2.609629e-03,True


## The Wire

In [38]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_The_Wire.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/the_wire_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_00", "Bubbles")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_01", "Cedric")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_02", "Proposition_Joe")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_03", "D'Angelo/Preston/Malik")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_05", "various_songs")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_06", "Ervin")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_07", "Stinger")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_09", "Lester")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_10", "Rhonda/Marla")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_11", "Moreland")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_12", "William_Rawls")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_13", "Ellis_Carver/Ziggy")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_14", "Herc")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_15", "Frank_Sobotka")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_16", "Omar")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_17", "Rhonda")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_20", "D'Angelo")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_22", "Roland")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_23", "Kima/Marla")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_24", "McNulty")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_25", "Maurice")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_26", "Avon/Wee-Bey")
  return df



df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [39]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

27

In [55]:
rho_stars

,rating,pval,significant?
rating,1.000000,1.547064e-275,True
Lester,0.508829,1.300342e-03,True
Ellis_Carver/Ziggy,-0.475423,2.941871e-03,True
various_songs,-0.380552,2.014975e-02,True
SPEAKER_04,0.305417,6.603132e-02,False
Moreland,-0.304941,6.647546e-02,False
Proposition_Joe,0.301969,6.930490e-02,False
SPEAKER_18,-0.287465,8.449325e-02,False
SPEAKER_08,-0.282353,9.042054e-02,False
Omar,0.272010,1.033858e-01,False


In [56]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,1.547064e-275,True
D'Angelo,0.605737,7.106871e-05,True
SPEAKER_19,-0.529872,7.440942e-04,True
Proposition_Joe,-0.497629,1.725490e-03,True
Roland,-0.497155,1.745912e-03,True
Avon/Wee-Bey,-0.397582,1.481847e-02,True
McNulty,0.392840,1.616710e-02,True
SPEAKER_21,-0.391181,1.666285e-02,True
Bubbles,0.330251,4.590163e-02,True
Maurice,0.286629,8.544153e-02,False


## The Punisher

In [40]:
df = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Clustered_final/Processed/Processed_The_Wire.csv")
ratings = pd.read_csv("/home/ada/Documents/DTA/Thesis/Data/Ratings_TV-shows/the_wire_ratings.csv")

def process(df):
  df['Speaker'] = df['Speaker'].astype(str)
  df['Speaker'] = df['Speaker'].apply(lambda x: "SPEAKER_" + x if len(x) == 2 else "SPEAKER_0" + x)
  return df

def name_all(df):
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_08", "Krista")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_11", "Frank")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_14", "Billy")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_15", "Eliza")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_19", "Dinah")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_20", "John")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_25", "Amy")
  df['Speaker'] = df['Speaker'].str.replace("SPEAKER_26", "Anderson")
  return df



df2 = process(df).copy()
df2 = name_all(df2)
speaker_list = df2['Speaker'].tolist()

rho_stars, rho_votes = get_correlations(df2, ratings)

/tmp/ipykernel_79152/3944648706.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['pval'] = [pvald_stars[t] for t in app_stars.index]
/tmp/ipykernel_79152/3944648706.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_stars['significant?'] = [True if value < 0.05 else False for value in app_stars.pval.values]


In [41]:
# Print total number of speakers tagged by the model

len(df.Speaker.unique())

27

In [4]:
rho_stars

,rating,pval,significant?
rating,1.000000,1.547064e-275,True
SPEAKER_09,0.508829,1.300342e-03,True
SPEAKER_13,-0.475423,2.941871e-03,True
SPEAKER_05,-0.380552,2.014975e-02,True
SPEAKER_04,0.305417,6.603132e-02,False
Frank,-0.304941,6.647546e-02,False
SPEAKER_02,0.301969,6.930490e-02,False
SPEAKER_18,-0.287465,8.449325e-02,False
Krista,-0.282353,9.042054e-02,False
SPEAKER_16,0.272010,1.033858e-01,False


In [5]:
rho_votes

,total_votes,pval,significant?
total_votes,1.000000,1.547064e-275,True
John,0.605737,7.106871e-05,True
Dinah,-0.529872,7.440942e-04,True
SPEAKER_02,-0.497629,1.725490e-03,True
SPEAKER_22,-0.497155,1.745912e-03,True
Anderson,-0.397582,1.481847e-02,True
SPEAKER_24,0.392840,1.616710e-02,True
SPEAKER_21,-0.391181,1.666285e-02,True
SPEAKER_00,0.330251,4.590163e-02,True
Amy,0.286629,8.544153e-02,False


In [ ]:
# rho stars average significant characters = 2.5 (2+2+3+3 /4)
# rho votes average significant characters = 9 (10+10+8+8 /4)